<a href="https://colab.research.google.com/github/mehrn79/npl_wrod2wec/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install hazm

     |████████████████████████████████| 316 kB 5.1 MB/s 
     |████████████████████████████████| 1.4 MB 41.6 MB/s 
     |████████████████████████████████| 233 kB 52.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=ecee5422cfaccb701b72a03950da50233cfade0d5759d16e9f2313a3a05624f0
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154488 sha256=402ddfa9d65fdbb069e3bd1919326c2a750a3c2ff31ac764d2777ac42b62e247
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [14]:
from __future__ import unicode_literals
from hazm import *
from keras.preprocessing import text

In [2]:
with open('/content/shams.txt') as f:
  lines = f.readlines()

with open('/content/stopwords.txt') as file:
  stopLines = file.readlines()
stopWord = [item.replace('\n',"") for item in stopLines]
  

In [3]:
from collections import Counter

words= []
stemmer = Stemmer()
for sent in lines :
  words.append(word_tokenize(sent))

text_word_list=[]
corpes=[]
new_words=[]

for wordSent in words :
  cleanSent=[]
  for word in wordSent :
    if word not in stopWord :
      cleanSent.append(stemmer.stem(word))
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    new_words.append(cleanSent)

value_of_repetition = dict(Counter(text_word_list))
repetition=[]
for word in value_of_repetition :
  if value_of_repetition.get(word)==1:
    repetition.append(word)
clear_word_list=[x for x in text_word_list if x not in repetition]

text_word_list=[]

for wordSent in new_words :
  cleanSent=[]
  for word in wordSent :
    if word not in repetition :
      cleanSent.append(word)
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    corpes.append(' '.join(cleanSent))

In [15]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpes)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

In [4]:
window = 2
word_lists = []
#text_word_list=[]

for text in corpes:
    text=word_tokenize(text)
    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])
len(word_lists)

110306

In [5]:
words_set = list(set(text_word_list))
words_set.sort()

unique_word_dict = {}
for i, word in enumerate(words_set):
  unique_word_dict.update({
    word: i
  })

#unique_word_dict
len(unique_word_dict)

3447

In [6]:
from scipy import sparse
import numpy as np
from tqdm import tqdm

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1
    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into an array
X = np.asarray(X)
Y = np.asarray(Y)


110306it [00:05, 20719.48it/s]


In [ ]:
# Deep learning: 
from keras.models import Input, Model
from keras.layers import Dense

# Defining the size of the embedding
embed_size = 100

# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam')

# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    epochs=100,
    verbose=2
    )



In [ ]:
#Obtaining the weights from the neural network. 
#These are the so called word embeddings

#The input layer 
weights = model.get_weights()[0]

#Creating a dictionary to store the embeddings in. The key is a unique word and 
#the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })
    
embedding_dict   

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:15]+1] 
                   for search_term in ['گنج','خدا','دنیا']}

similar_words